## using sqlalchemy to get joined json format data

In [2]:
# Import the dependencies
import sqlalchemy
import pandas as pd
from flask import Flask, jsonify

# Python SQL toolkit and Object Relational Mapper
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect
from pathlib import Path


In [4]:
# Database Setup

engine = create_engine("sqlite:///../data/Obesity_db.sqlite")

# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(autoload_with=engine)

#test view all classes found by automap
Base.classes.keys()


[]

In [5]:
conn = engine.connect()

#populate obesity table
Obesity = pd.read_sql("SELECT * FROM Obesity", conn)

In [19]:
Obesity

,State,Prevalence,95% CI
0,Alabama,39.9,"(37.9, 41.8)"
1,Alaska,33.5,"(31.5, 35.5)"
2,Arizona,31.3,"(30.0, 32.6)"
3,Arkansas,38.7,"(36.6, 40.8)"
4,California,27.6,"(26.1, 29.1)"
...,...,...,...
104,Virginia,34.2,"(32.9, 35.6)"
105,Washington,28.8,"(27.7, 29.9)"
106,West Virginia,40.6,"(39.1, 42.1)"
107,Wisconsin,33.9,"(32.1, 35.7)"


In [18]:
#populate fastfood table
Fastfood = pd.read_sql("SELECT * FROM Fastfood", conn)
Fastfood

,State,All fast food restaurants,Full-service restaurants,Subway,Starbucks,McDonalds,Dunkin Donut,Burger King,Taco Bell,Dominos,Wendys,Dairy Queen
0,Alabama,81.7,57.6,8.2,1.8,5.0,0.9,3.5,2.8,2.5,2.0,1.3
1,Alaska,61.9,78.7,7.2,6.6,4.2,0.3,1.1,2.1,1.4,1.2,0.7
2,Arizona,67.9,59.2,5.6,7.3,3.9,1.3,2.5,2.5,1.6,1.4,1.5
3,Arkansas,69.9,67.6,8.4,2.0,5.7,0.3,2.7,3.5,2.2,2.1,1.0
4,California,82.3,80.4,5.7,7.4,3.2,0.3,1.4,2.1,1.4,0.7,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...
96,Virginia,80.1,81.7,7.3,5.2,4.7,2.8,2.3,2.4,2.6,2.6,1.1
97,Washington,66.8,85.1,6.8,9.7,3.4,0.0,1.6,1.9,2.0,1.0,1.3
98,West Virginia,73.8,66.2,9.7,2.2,5.8,1.3,3.5,3.5,2.3,3.9,3.9
99,Wisconsin,65.0,86.4,9.1,3.0,5.1,1.2,2.1,2.3,1.8,0.9,2.2


In [8]:
#verify tables in engine
inspector = inspect(engine)
inspector.get_table_names()

['fastfood', 'obesity']

In [12]:
#create session from Python to the DB
session = Session(engine)

In [13]:
#inner join for data cleanup
joined_table = pd.read_sql('Select fastfood.State, fastfood."All fast food restaurants", fastfood.Subway, fastfood.Starbucks, fastfood.McDonalds, fastfood."Dunkin Donut", fastfood."Burger King", fastfood."Taco Bell", fastfood.Dominos, fastfood.Wendys, fastfood."Dairy Queen", obesity.Prevalence, obesity."95% CI" from Fastfood INNER JOIN Obesity ON (Fastfood.State = Obesity.State)', conn)

In [15]:
#preview data in joined_table
joined_table

,State,All fast food restaurants,Subway,Starbucks,McDonalds,Dunkin Donut,Burger King,Taco Bell,Dominos,Wendys,Dairy Queen,Prevalence,95% CI
0,Alabama,81.7,8.2,1.8,5.0,0.9,3.5,2.8,2.5,2.0,1.3,39.9,"(37.9, 41.8)"
1,Alabama,81.7,8.2,1.8,5.0,0.9,3.5,2.8,2.5,2.0,1.3,39.9,"(37.9, 41.8)"
2,Alaska,61.9,7.2,6.6,4.2,0.3,1.1,2.1,1.4,1.2,0.7,33.5,"(31.5, 35.5)"
3,Alaska,61.9,7.2,6.6,4.2,0.3,1.1,2.1,1.4,1.2,0.7,33.5,"(31.5, 35.5)"
4,Arizona,67.9,5.6,7.3,3.9,1.3,2.5,2.5,1.6,1.4,1.5,31.3,"(30.0, 32.6)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,West Virginia,73.8,9.7,2.2,5.8,1.3,3.5,3.5,2.3,3.9,3.9,40.6,"(39.1, 42.1)"
197,Wisconsin,65.0,9.1,3.0,5.1,1.2,2.1,2.3,1.8,0.9,2.2,33.9,"(32.1, 35.7)"
198,Wisconsin,65.0,9.1,3.0,5.1,1.2,2.1,2.3,1.8,0.9,2.2,33.9,"(32.1, 35.7)"
199,Wyoming,62.5,10.0,4.3,4.8,0.2,2.8,2.4,3.8,2.4,1.6,32,"(29.9, 34.1)"


In [17]:
fixed_joined_table = joined_table.drop_duplicates()
fixed_joined_table

,State,All fast food restaurants,Subway,Starbucks,McDonalds,Dunkin Donut,Burger King,Taco Bell,Dominos,Wendys,Dairy Queen,Prevalence,95% CI
0,Alabama,81.7,8.2,1.8,5.0,0.9,3.5,2.8,2.5,2.0,1.3,39.9,"(37.9, 41.8)"
2,Alaska,61.9,7.2,6.6,4.2,0.3,1.1,2.1,1.4,1.2,0.7,33.5,"(31.5, 35.5)"
4,Arizona,67.9,5.6,7.3,3.9,1.3,2.5,2.5,1.6,1.4,1.5,31.3,"(30.0, 32.6)"
6,Arkansas,69.9,8.4,2.0,5.7,0.3,2.7,3.5,2.2,2.1,1.0,38.7,"(36.6, 40.8)"
8,California,82.3,5.7,7.4,3.2,0.3,1.4,2.1,1.4,0.7,0.2,27.6,"(26.1, 29.1)"
10,Colorado,75.7,6.7,8.8,3.6,0.8,2.0,2.6,2.3,2.3,1.5,25.1,"(24.1, 26.2)"
12,Connecticut,76.1,8.1,3.4,4.0,14.4,1.8,1.4,1.5,1.5,1.1,30.4,"(28.9, 31.9)"
14,Delaware,78.3,2.5,3.6,3.7,7.1,2.1,1.5,2.7,1.2,1.5,33.9,"(31.8, 36.1)"
16,Florida,65.5,6.0,3.6,4.2,4.1,2.7,2.1,1.9,2.5,0.7,No data,No data
18,Georgia,82.5,7.1,3.6,4.2,2.2,2.6,2.3,2.1,2.7,2.0,33.9,"(32.2, 35.6)"


In [ ]:
#convert to json
json_table = joined_table.to_json()